In [25]:
# Week 5 
# Author : P. Mondal
# Date : Oct 20, 2020

In [1]:
import numpy as np

In [2]:
# Function to get short rate tree
def ShortRateTree(r00, qu, qd, n):
    shortRtree = np.zeros((n+1, n+1))
    
    # Setting r00 at i = 0 and j = 0
    shortRtree[0, 0] = r00
    for i in range(1, n+1):
      shortRtree[i, 0] = qd*shortRtree[i-1, 0] # ???? the code sol used qu 
      for j in range(1, n+1):
        shortRtree[i, j] = qu*shortRtree[i-1, j-1]
    return shortRtree



In [3]:
# Reproduced shortrate tree from class slides
test_shorttree = ShortRateTree(6, 1.25, 0.9, 5)
print(test_shorttree)

[[ 6.          0.          0.          0.          0.          0.        ]
 [ 5.4         7.5         0.          0.          0.          0.        ]
 [ 4.86        6.75        9.375       0.          0.          0.        ]
 [ 4.374       6.075       8.4375     11.71875     0.          0.        ]
 [ 3.9366      5.4675      7.59375    10.546875   14.6484375   0.        ]
 [ 3.54294     4.92075     6.834375    9.4921875  13.18359375 18.31054688]]


In [4]:
# Function to generate bond tree
def bondtree(bondMtry, r00, F, qu, qd, n):
  """Generates zero coupon bond tree"""
  q1 = q2 = 0.5 # Not sure 
  # Get shortrate tree corresponding to r00, qu, qd, n
  shortrT = ShortRateTree(r00, qu, qd, n)
  # print("shortT is\n", shortrT)

  bondT = np.zeros((bondMtry+1, bondMtry+1))
  # Generate bond tree : Going backward here
  for j in range(bondMtry+1):
    # At maturity all node states are set to facevalue
    bondT[bondMtry, j] = F
  # print(bondT)

  for i in range(bondMtry-1, -1, -1):
    for j in range(i+1):
      # Each i^th time has j = i + 1 nodes
      # print(i, j, bondT[i, j])
      bondT[i, j] = (1/(1+shortrT[i, j]))*(q1*bondT[i+1, j+1]
                                         + q2*bondT[i+1, j])
  return bondT


In [5]:
# Produced result from slide 
bondMtry = 4 # Bond maturity time
r00 = 6/100 # spot intereat rate at i = 0, j = 0
F = 100 # Facevalue 
qu = 1.25 # short interest rate up
qd = 0.9 # short interest rate down
n = 7 # number o short rate periods
bnd_tree = bondtree(bondMtry, r00, F, qu, qd, n)

In [6]:
print("Zero coupon price is ", bnd_tree[0, 0])

Zero coupon price is  77.217740328716


In [7]:
# American call option on the same ZCB 
def bondOpAm(r00, qu, qd, F, n, bondMtry, K, N, cp):
  # cp : +1/-1 with regards to call/put

  q1 = q2 = 0.5
  # Short rate tree
  shortrT = ShortRateTree(r00, qu, qd, n)
  f = np.vectorize(lambda x:max(float(x) - float(K), 0.))
  bond_Tr = bondtree(bondMtry, r00, F, qu, qd, n)

  payoff = f(bond_Tr)
  option_Tr = np.zeros((N+1, N+1))
  # print("short tree\n", shortrT)

  # Compute the option tree
  flag = 0
  list = []
   # Pay off at t = N 
  for j in range(N+1):
    option_Tr[N, j] = max(0, cp*(bond_Tr[N, j]-K))

  for i in range(N-1, -1, -1):
    for j in range(i+1):
      # print('bnvv', i, j, payoff[i, j], payoff[i+1, j], payoff[i+1, j+1], q1,shortrT[i, j] )
      option_Tr[i, j] = max((q1*option_Tr[i+1, j] + q2*option_Tr[i+1, j+1])
                            *1/(1+shortrT[i, j]), 
                             cp*(bond_Tr[i, j]-K ))
      
      # Early exercise
      if (option_Tr[i, j] - cp*(bond_Tr[i, j]-K)) < 1e-10:
        flag +=1
        list.append(i)

  when = N
  if flag:
    when = list[-1]
  # print('Option tree\n', payoff)
  return (option_Tr[0, 0], when)

In [8]:
def bondForwrd(N, r00, qu, qd, F, bondMtry):
  """Looks like is the the ratio of price of considering bond maturity 
     and derivative maturity 
  """
  down = bondtree(N, r00, 1, qu, qd, n)
  up = bondtree(bondMtry, r00, F, qu, qd, n)
  # print('down is computed correctly,', down[0, 0])
  # print('up \n', up)
  
  # print(up)
  return up[0, 0]/down[0, 0]

In [9]:
N_for = 4 # This is correct --> produced right down value
bondMtry = 6 
F = 110
print("Price of Forward at t=0 :\n", bondForwrd(N_for, r00, qu, qd, F, bondMtry))
# I think the slide calculated some wrong lattice points starting i = N

Price of Forward at t=0 :
 94.10815111093011


In [10]:
# GenerateBondTree(n, bondMtry, face, r00, u, d):
# me bondtree(bondMtry, r00, F, qu, qd, n)

def bondFuture(N, bondMtry, r00, F, qu, qd, n):
  """Computes price of future bond on ZCB"""
  # Computte bond tree till bond maturity i.e. bondMtry
  bond_Tr = bondtree(bondMtry, r00, F, qu, qd, n)

  q1 = q2 = 0.5
  # Compute futures tree
  futures_Tr = np.zeros((N+1, N+1))
  for j in range(N+1):
    futures_Tr[N, j] = bond_Tr[N, j]
  
  for i in range(N-1, -1, -1):
    for j in range(i+1):
      futures_Tr[i, j] = q1*futures_Tr[i+1, j]\
                       + q2*futures_Tr[i+1, j+1]
  print('futures price using ZCB\n')
  return futures_Tr[0, 0]  

In [11]:
# Elementary pricing
# What is elementary ??
def getElementary(r00, qu, qd, n, N):
  """Elementary pricing"""
  shortRTr = ShortRateTree(r00, qu, qd, n)
  elemTr = np.zeros((N+1, N+1))

  elemTr[0, 0] = 1
  for j in range(1, N+1):
    elemTr[j, j] = 0.5/(1+shortRTr[j-1, j-1])*elemTr[j-1, j-1]
  for i in range(1, N+1):
    elemTr[i, 0] = 0.5/(1+shortRTr[i-1, 0])*elemTr[i-1, 0] 
  for i in range(2, N+1):
    for j in range(1, i):
      elemTr[i, j] = 0.5/(1+shortRTr[i-1, j])*elemTr[i-1, j] + \
                    0.5/(1+shortRTr[i-1, j-1])*elemTr[i-1, j-1]
  return elemTr
  # Note : Look up the recursive formula used here ????


In [ ]:
N_elm = 6
print(getElementary(r00, qu, qd, n, N_elm))

[[1.         0.         0.         0.         0.         0.
  0.        ]
 [0.47619048 0.47619048 0.         0.         0.         0.
  0.        ]
 [0.22784233 0.45352502 0.22568269 0.         0.         0.
  0.        ]
 [0.10948695 0.32555413 0.32247109 0.10640391 0.         0.
  0.        ]
 [0.05281825 0.20865288 0.30874425 0.20279191 0.04988229 0.
  0.        ]
 [0.02557029 0.12587501 0.24746525 0.24282656 0.11890591 0.02323987
  0.        ]
 [0.0124185  0.07316397 0.17925145 0.23370917 0.17098124 0.06653201
  0.01075397]]


In [12]:
def forward_equation_lattice(n,r00,u,d,q=.5):
    tree = np.zeros((n+1,n+1))
    tree[0][0]=1
    rate_tree= ShortRateTree(r00, u, d, n) 
    for i in range(1,n+1):
        tree[i][0]=1/(1+rate_tree[i-1][0])*(1-q)*tree[i-1][0]
        tree[i][i]=1/(1+rate_tree[i-1][i-1])*q*tree[i-1][i-1]
    for i in range(2,n+1):
        for j in range(1,i):
            tree[i,j]=(1-q)*1/(1+rate_tree[i-1,j])*tree[i-1,j]+q*1/(1+rate_tree[i-1,j-1])*tree[i-1,j-1]
    return tree

In [13]:
forward_equation_lattice(n,r00,qu,qd,q=.5)

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.47169811, 0.47169811, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.22376571, 0.44316018, 0.21939447, 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.10669736, 0.31426653, 0.30786379, 0.10029462, 0.        ,
        0.        , 0.        , 0.        ],
       [0.051113  , 0.19924712, 0.2900886 , 0.18684158, 0.0448871 ,
        0.        , 0.        , 0.        ],
       [0.02458855, 0.11904756, 0.22926638, 0.21931522, 0.10408383,
        0.01957598, 0.        , 0.        ],
       [0.0118736 , 0.06860573, 0.16403204, 0.20745099, 0.14613117,
        0.05425322, 0.00827313, 0.        ],
       [0.00575335, 0.03860147, 0.11011172, 0.17282528, 0.16087741,
        0.08860446, 0.02665486, 0.00336612]])

In [14]:
def forward_starting_swap_price(n,r00,u,d,q=.5,notional_principal = 1000000.0,fixed_rate = 0.045):
    lattice = forward_equation_lattice(n,r00,u,d,q=.5)
    rate_tree= ShortRateTree(r00, u, d, n)
    s = 0 
    for i in range(1,n+1):
        for j in range(i+1):
            s += -(fixed_rate - rate_tree[i][j])/(1+rate_tree[i][j])*lattice[i,j]
    return s*notional_principal



In [16]:
# --------
# Quiz 1 :
# --------
bondMtry = 10
r00 = 5/100
F = 100 
qu = 1.1
qd = 0.9
n = bondMtry + 2 # Getting shortrate tree for two more bond periods
bnd_tree_q1 = bondtree(bondMtry, r00, F, qu, qd, n)
print("Zero coupon price is ", bnd_tree_q1[0, 0])

Zero coupon price is  61.62195811754156


In [17]:
# ------
# Quiz 2
# ------
N_for = 4
print("Price of Forward at t=0 :\n", bondForwrd(N_for, r00, qu, qd, F, 
                                                              bondMtry))

Price of Forward at t=0 :
 74.88484493844841


In [18]:
# ------
# Quiz 3
# ------
N_fut = 4
# bondFuture(N, bondMtry, r00, F, qu, qd, n)
print("Price of Forward at t=0 :\n", bondFuture(N_fut, bondMtry, r00, F, qu, qd, n))

futures price using ZCB

Price of Forward at t=0 :
 74.82458063139569


In [19]:
# ------
# Quiz 4
# ------
cp = 1 # call option
K = 80
N_bond = 6 
print("American call option is ", bondOpAm(r00, qu, qd, F, n, bondMtry, K, N_bond, cp))


American call option is  (2.35721516382906, 6)


In [20]:
# ------
# Quiz 5
# ------
# TODO ????
rf = 0.045
start = 1
pcp1 = 10**6
n = 10
rp = 1 # pay
print("Swap option price is ",forward_starting_swap_price(n,r00,qu,qd,q=.5,
                           notional_principal = 1000000.0,fixed_rate = 0.045))


Swap option price is  33374.242062163816


In [21]:
def swap_price_lattice(n,r00,u,d,q=.5,notional_principal = 1000000.0,fixed_rate = 0.045):
    rate_tree= ShortRateTree(r00, u, d, n)  # short_rate_tree(n,r00,u,d)
    payoff = np.zeros((n+1,n+1))
    for i in range(n+1):
        payoff[n,i]=(rate_tree[n,i]-fixed_rate)/(1+rate_tree[n,i])
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            payoff[i,j]= (rate_tree[i,j]-fixed_rate+q*payoff[i+1,j+1]+(1-q)*payoff[i+1,j])/(1+rate_tree[i,j])
    return payoff



In [22]:
swap_price_lattice(n,r00,qu,qd,q=.5,fixed_rate = 0.045);

In [23]:
def swaption_price(n,r00,u,d,q=.5,notional_principal = 1000000.0,fixed_rate = 0.045,t=5,K=0.0):
    lattice = swap_price_lattice(n,r00,u,d,q=.5,fixed_rate = 0.045)
    rate_tree= ShortRateTree(r00, u, d, n) # short_rate_tree(n,r00,u,d)
    f= np.vectorize(lambda x: max(x-K,0.0))
    payoff = f(lattice)[0:t+1,0:t+1]
    for i in range(t-1,-1,-1):
        for j in range(i+1):
            payoff[i,j]=(q*payoff[i+1,j+1]+(1-q)*payoff[i+1,j])/(1+rate_tree[i,j])
    return payoff[0,0]*notional_principal

In [24]:
K = 0
pcp1 = 1000000
N=5
rf = 0.045

n= 10 #10 period
r00= 0.05
qu=1.1
qd=0.9
q=.5
face_value=100.0

# Swaption(n, N, pcp1, K, r00, rf, qu, qd, rp=1, start=0)
# Swaption(n, N_swap=5, pcp1=1000000.0, K = 0.0, r00, rf=0.045, qu, qd, rp=1, start=1)
print('Swaption pricd is\n', swaption_price(n,r00,qu,qd,q=.5,
                  notional_principal = 1000000.0,fixed_rate = 0.045,t=5,K=0.0))

Swaption pricd is
 26311.079490192285
